In [26]:
import librosa as lb
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
location = "D:/dnn_data/"
duration = 2
sampling_rate = 44100
total_length = duration * sampling_rate

# 데이터 추출(chroma, mfcc)
def featureExtract(file_location):
    #샘플링된 주파수 로드
    data, sr = lb.core.load(file_location, sr=sampling_rate, mono=True, res_type="kaiser_fast") 
    # 파일이 일정이상 커지면 그 뒤 값은 모두 랜덤
    if len(data)> total_length:
        max_offset = len(data) - total_length
        offset = np.random.randint(max_offset)
        data = data[offset:(total_length+offset)]
    else:
        if total_length > len(data):
            max_offset = total_length - len(data)
            offset = np.random.randint(max_offset)
        else:
            offset = 0
        data = np.pad(data, (offset, total_length - len(data) - offset), "constant")
        
    data = np.mean(lb.feature.chroma_stft(data, sr=sampling_rate, n_chroma=40),axis=1)
    # mfcc = np.mean(lb.feature.mfcc(data, sr=sampling_rate n_mfcc=40).T,axis=0)
    # return chroma_stft,mfcc
    return data

#데이터 로드
train = pd.read_csv(location+"audio_train.csv")

#인덱스 설정(label에 각각 숫자부여)
label_index = pd.DataFrame(columns=("labeled_num","label"))
label_index["labeled_num"] = range(len(train["label"].unique()))
label_index["label"] = train['label'].unique()
train=pd.merge(train, label_index, how="left")

# 숫자와 파일이름 빼고 제거
train = train.drop(['label','manually_verified'],axis=1)
trainFname = train.values[:,0]

#chroma_data 초기화
rows=len(train)
chroma_data = np.zeros((rows,41))


array([[-4.52890081e+02, -4.56222687e+02, -4.66719986e+02, ...,
        -3.16716329e+02, -3.14714728e+02, -3.51715271e+02],
       [ 1.09764818e+02,  1.10151123e+02,  1.02906970e+02, ...,
         1.76346687e+02,  1.71322634e+02,  1.69911369e+02],
       [-1.02775166e+02, -1.07260834e+02, -1.04937068e+02, ...,
        -1.40038156e+02, -1.38895233e+02, -1.14569420e+02],
       ...,
       [-6.73978383e-01, -4.52522730e+00, -6.75935167e+00, ...,
         1.54524589e+01,  1.79918237e+01,  1.24936325e+01],
       [ 3.46630530e+00,  4.33823578e+00,  3.81609180e+00, ...,
         2.29620594e+01,  1.63315878e+01,  7.16992160e+00],
       [ 2.57866977e-01,  2.86196174e+00,  2.20079275e+00, ...,
         1.28499487e+01,  1.98989673e+00, -1.72018730e+00]])

In [7]:

# 데이터 전체 chroma-stft
i=0
for fname in trainFname:
    labeled_num=train.loc[train['fname']==fname,"labeled_num"]
    chroma_data[i] = np.hstack([labeled_num,featureExtract(location+"/audio_train/"+fname)])
    i+=1


C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [29]:
# train, test 데이터 분리
train_set, test_set = train_test_split(chroma_data, test_size=0.3)

"""
train_label = train_set.values[:,1]
test_label = test_set.values[:,1]
train.head()
"""
train_set

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
# convoiution


array([[ 5.        ,  0.6623496 ,  0.63130314, ...,  0.76465069,
         0.73147868,  0.69762005],
       [23.        ,  0.33043352,  0.35030964, ...,  0.44970445,
         0.44460435,  0.3622034 ],
       [21.        ,  0.65610325,  0.68185873, ...,  0.67387157,
         0.66327612,  0.66089417],
       ...,
       [13.        ,  0.54429821,  0.52198626, ...,  0.54383328,
         0.55144017,  0.54626469],
       [ 4.        ,  0.19783503,  0.22793309, ...,  0.1580327 ,
         0.17532297,  0.18381838],
       [ 2.        ,  0.42361788,  0.40207798, ...,  0.55458859,
         0.51958872,  0.4757151 ]])